In [1]:
import pandas as pd
import numpy as np


In [2]:
import sqlite3

In [ ]:
#обновляю версию sqlite, чтобы включить поддержку оконных функций
!gdown --id 1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
!mv _sqlite3.cpython-37m-x86_64-linux-gnu.so /usr/lib/python3.7/lib-dynload/
import os
os.kill(os.getpid(), 9)

Downloading...
From: https://drive.google.com/uc?id=1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
To: /content/_sqlite3.cpython-37m-x86_64-linux-gnu.so
100% 6.33M/6.33M [00:00<00:00, 37.3MB/s]


In [3]:
sqlite3.sqlite_version

'3.36.0'

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [49]:
#cоздаю базу данных
connect = sqlite3.connect('/content/gdrive/My Drive/database_flights.db')

In [7]:
#подключаю локально сгенерированные файлы
flights = pd.read_csv('/content/gdrive/My Drive/flights.csv')
planes = pd.read_csv('/content/gdrive/My Drive/planes.csv')
pilots = pd.read_csv('/content/gdrive/My Drive/pilots.csv')

In [17]:
#предврарительно сгенерировано 1399 полетов
flights.head()

,flight_id,flight_dt,plane_id,first_pilot_id,second_pilot_id,destination,quantity
0,1,2021-08-31 0:28:48,220,2,111,Грабцево,243
1,2,2021-06-21 8:49:42,404,109,86,Чертовицкое,237
2,3,2021-11-17 23:11:43,787,11,114,Брянск,179
3,4,2021-09-09 16:30:54,727,111,100,Братск,63
4,5,2021-10-28 13:07:38,18,10,114,Иркутск,44


In [16]:
#19 разных самолетов
planes.head()

,plane_id,capacity,cargo_flg
0,747,20,1
1,737,25,1
2,727,35,1
3,777,30,1
4,320,20,1


In [19]:
#и 122 пилота
pilots.head()

,pilot_id,name,age,rank,education_level
0,1,Жуков Иван Матвеевич,43,1 category,1
1,2,Ермаков Герман Максимович,47,9 category,2
2,3,Кузнецов Тимофей Романович,47,10 category,4
3,4,Карасева Сафия Александровна,38,7 category,3
4,5,Молчанов Марк Вячеславович,24,1 category,3


In [20]:
#Дата отображается неправильно
flights.dtypes

flight_id           int64
flight_dt          object
plane_id            int64
first_pilot_id      int64
second_pilot_id     int64
destination        object
quantity            int64
dtype: object

In [21]:
#Поменяем тип даты на определенный формат
flights['flight_dt'] = pd.to_datetime(flights['flight_dt'],format='%Y-%m-%d %H:%M:%S')

In [22]:
flights.dtypes

flight_id                   int64
flight_dt          datetime64[ns]
plane_id                    int64
first_pilot_id              int64
second_pilot_id             int64
destination                object
quantity                    int64
dtype: object

In [50]:
flights.to_sql('flights',connect,index=False,if_exists='replace')
planes.to_sql('planes',connect,index=False,if_exists='replace')
pilots.to_sql('pilots',connect,index=False,if_exists='replace')

In [24]:
def select(sql):
  return pd.read_sql(sql,connect)

In [51]:
#создаем сводную таблицу, где каждый полет дублируется для каждого пилота (первого и второго) и параллельно присоединяем информацию о самолете
cur = connect.cursor()
sql = '''
DROP TABLE IF EXISTS agg_table;

CREATE TABLE agg_table AS
SELECT f.flight_id,
       f.flight_dt,
       f.first_pilot_id,
       f.second_pilot_id,
       f.destination,
       f.quantity,
       p.pilot_id,
       p.name,
       p.age,
       pl.plane_id,
       pl.cargo_flg
FROM flights AS f
JOIN pilots AS p ON p.pilot_id = f.first_pilot_id
OR p.pilot_id = f.second_pilot_id
JOIN planes AS pl ON pl.plane_id = f.plane_id
'''
cur.executescript(sql)

Задача 1. Напишите запрос, который выведет пилотов, которые в качестве второго пилота в
августе этого года ездили в аэропорт Шереметьево. 

In [34]:
sql = '''
SELECT DISTINCT t.pilot_id,
                t.name
FROM agg_table AS t
WHERE t.second_pilot_id = t.pilot_id
  AND t.flight_dt BETWEEN '2021-08-01 00:00:00' AND '2021-08-31 23:59:59'
  AND t.destination = 'Шереметьево'
ORDER BY t.pilot_id
'''
select(sql)

,pilot_id,name
0,1,Жуков Иван Матвеевич
1,13,Цветков Тимур Ярославович
2,16,Смирнов Павел Максимович
3,18,Овчинников Михаил Александрович
4,22,Афанасьев Артём Егорович
5,31,Пономарев Тимофей Максимович
6,32,Мартынов Александр Семёнович
7,38,Игнатьев Глеб Алексеевич
8,39,Соколов Михаил Максимович
9,41,Быков Александр Тимофеевич


Задача 2. Выведите пилотов старше 45 лет, которые совершили больше 30 полетов на грузовых самолетах.

In [52]:
sql = '''
SELECT t.pilot_id,
       t.name
FROM agg_table AS t
WHERE t.age > 45
  AND t.cargo_flg = 1
GROUP BY t.name,
         t.pilot_id
HAVING count(1) > 30
ORDER BY t.pilot_id
'''

select(sql)

,pilot_id,name
0,2,Ермаков Герман Максимович
1,3,Кузнецов Тимофей Романович
2,9,Селиванов Константин Ильич
3,12,Клюева Василиса Степановна
4,32,Мартынов Александр Семёнович
5,66,Фомин Олег Дмитриевич


Задача 3. Выведите ТОП 10 пилотов-капитанов (first_pilot_id), которые перевезли наибольшее
число пассажиров в этом году.

In [48]:
sql = '''
WITH sum_table AS
  (SELECT t.pilot_id,
          t.name,
          sum(t.quantity) AS quantity
   FROM agg_table AS t
   WHERE t.first_pilot_id = t.pilot_id
     AND t.cargo_flg = 0
     AND t.flight_dt BETWEEN '2021-01-01 00:00:00' AND '2021-12-31 23:59:59'
   GROUP BY t.pilot_id),

     rankings AS
  (SELECT s.*,
          row_number() OVER (ORDER BY s.quantity DESC) AS rnk
   FROM sum_table AS s)
   
SELECT r.pilot_id,
       r.name,
       r.quantity
FROM rankings AS r
WHERE r.rnk <=10
'''
select(sql)

,pilot_id,name,quantity
0,96,Никонов Иван Матвеевич,1619
1,103,Голованова Александра Михайловна,1525
2,97,Агеев Илья Фёдорович,950
3,89,Колосов Дмитрий Глебович,929
4,50,Суворов Матвей Миронович,918
5,82,Горохов Макар Алексеевич,856
6,32,Мартынов Александр Семёнович,843
7,11,Ушаков Даниил Даниилович,826
8,17,Щукина Маргарита Кирилловна,807
9,99,Иванова Анна Тимуровна,803
